# Code for AMAS project 1
### *By Niels August Davidsen*

In [45]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [83]:

# URL of the webpage
url = 'https://www.nbi.dk/%7Ekoskinen/Teaching/AdvancedMethodsInAppliedStatistics2025/data/2014KenPomeroy.html'

# Send a GET request to the webpage
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the webpage content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table using soup (BS4)
table = soup.find('table', {'id': 'ratings-table'})

# Extract table headers
headers = [th.get_text() for th in table.find('thead').find_all('th')]
headers = list(filter(lambda x: x != '', headers))
headers = headers[2:]
print(headers)

# Finding strings with ONLY integers of length 1, 2 or 3
pattern = re.compile(r'^\d{1,3}$')
reg_city = re.compile(r'$\d+$')

# Extract table rows
rows = []
for row_number,row in enumerate(table.find('tbody').find_all('tr')):
    cells = row.find_all(['th', 'td'])
    cells_text = [cell.get_text() for cell in cells]

    if cells_text[0] == '' or cells_text[0] == ' ':
        continue
    
    # Remove elements in cells_text that concists ONLY of integers of length 1, 2 or 3
    for i, element in enumerate(cells_text):

        if pattern.match(element) and i > 1:
            cells_text.pop(i)

    print(repr(cells_text[1]))
    print(cells_text[1][:-2])
    matches = reg_city.match(cells_text[1])
    if matches is not None:
        cells_text[1] = cells_text[1][:-2]
        

    print(repr(cells_text[1]))


    # Creating an fail-safe for the case where the length of cells_text does not match the length of headers
    if len(cells_text) != len(headers):
        print(len(cells_text), len(headers))
        print(f'Error: Length of cells_text does not match length of headers in row {row_number}')
    
    rows.append(cells_text)



['Rk', 'Team', 'Conf', 'W-L', 'NetRtg', 'ORtg', 'DRtg', 'AdjT', 'Luck', 'NetRtg', 'ORtg', 'DRtg', 'NetRtg']
'Louisville 4'
Louisville
'Louisville 4'
'Arizona 1'
Arizona
'Arizona 1'
'Florida 1'
Florida
'Florida 1'
'Virginia 1'
Virginia
'Virginia 1'
'Wisconsin 2'
Wisconsin
'Wisconsin 2'
'Wichita St. 1'
Wichita St.
'Wichita St. 1'
'Kansas 2'
Kansas
'Kansas 2'
'Duke 3'
Duke
'Duke 3'
'Michigan St. 4'
Michigan St.
'Michigan St. 4'
'Tennessee 11'
Tennessee 
'Tennessee 11'
'Villanova 2'
Villanova
'Villanova 2'
'Michigan 2'
Michigan
'Michigan 2'
'Kentucky 8'
Kentucky
'Kentucky 8'
'UCLA 4'
UCLA
'UCLA 4'
'Connecticut 7'
Connecticut
'Connecticut 7'
'Pittsburgh 9'
Pittsburgh
'Pittsburgh 9'
'Creighton 3'
Creighton
'Creighton 3'
'Syracuse 3'
Syracuse
'Syracuse 3'
'Ohio St. 6'
Ohio St.
'Ohio St. 6'
'Iowa St. 3'
Iowa St.
'Iowa St. 3'
'Oklahoma St. 9'
Oklahoma St.
'Oklahoma St. 9'
'Iowa 11'
Iowa 
'Iowa 11'
'San Diego St. 4'
San Diego St.
'San Diego St. 4'
'Gonzaga 8'
Gonzaga
'Gonzaga 8'
'VCU 5'
VCU
'VCU

In [75]:
# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Save to CSV
df.to_csv('kenpom_2014_ratings.csv', index=False)

In [76]:
df

,Rk,Team,Conf,W-L,NetRtg,ORtg,DRtg,AdjT,Luck,NetRtg,ORtg,DRtg,NetRtg
0,1,Louisville 4,Amer,31-6,+30.41,118.9,88.5,67.2,-.045,+4.31,107.0,102.7,-4.65
1,2,Arizona 1,P12,33-5,+30.11,116.5,86.4,62.8,+.014,+9.30,109.9,100.6,+1.62
2,3,Florida 1,SEC,36-3,+28.57,116.5,88.0,61.3,+.053,+9.02,109.4,100.4,+2.39
3,4,Virginia 1,ACC,30-7,+26.46,114.8,88.3,59.5,+.012,+8.67,109.4,100.8,+1.57
4,5,Wisconsin 2,B10,30-8,+25.89,122.0,96.1,62.4,+.019,+11.44,110.2,98.8,+3.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,347,Mississippi Valley St.,SWAC,9-23,-22.93,93.0,115.9,70.9,+.012,-8.65,98.8,107.4,+6.21
355,348,Abilene Christian,Slnd,11-20,-23.35,94.0,117.3,63.2,-.045,-2.67,104.7,107.4,+13.38
356,349,Presbyterian,BSth,6-26,-25.87,94.2,120.0,63.5,-.017,-7.84,101.7,109.6,-7.05
357,350,Grambling St.,SWAC,5-24,-26.60,95.5,122.1,66.0,-.017,-9.05,98.7,107.8,+6.51
